In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

from torchvision import transforms
import torchvision

import medmnist
from medmnist import INFO, BreastMNIST, BloodMNIST

import matplotlib.pyplot as plt

import os

In [ ]:
dir = os.getcwd()
dir = os.path.dirname(dir)

data_flag = "bloodmnist"
data_dir = "Datasets"

PATH = os.path.join(dir, data_dir)

# retrieve the data from the 
datasets = {x: BloodMNIST(split= x, transform=None, download=False, root=PATH)
            for x in ["train", "val", "test"]} 
["train", "val", "test"]
images = {x: np.array(datasets[x].imgs) for x in ["train", "val", "test"]}
labels = {x: np.array(datasets[x].labels).flatten() for x in ["train", "val", "test"]}

In [ ]:
from skimage.feature import hog

def extract_hog(image):
    fd = hog(
        image,
        orientations=6,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        visualize=False,
        channel_axis=0,
    )
    return fd

In [ ]:
dir = os.getcwd()
dir = os.path.dirname(dir)

data_flag = "bloodmnist"
data_dir = "Datasets"

PATH = os.path.join(dir, data_dir)

data_transforms = {
            "train": transforms.Compose([
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.Lambda(lambda x: x.squeeze().numpy()),
                transforms.Lambda(lambda x: x.flatten()),
                # transforms.Lambda(extract_hog)
            ]),
            "val": transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(lambda x: x.squeeze().numpy()),
                transforms.Lambda(lambda x: x.flatten()),
                # transforms.Lambda(extract_hog)
            ]),
            "test": transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(lambda x: x.squeeze().numpy()),
                transforms.Lambda(lambda x: x.flatten()),
                # transforms.Lambda(extract_hog)
            ]),
        }

# retrieve the data from the 
datasets = {x: BloodMNIST(split= x, transform=data_transforms[x], download=False, root=PATH)
            for x in ["train", "val", "test"]} 

# get the size of all 3 data splits
dataset_sizes = {x: len(datasets[x]) for x in ["train", "val", "test"]}

# process the images
inputs = {x: np.array([datasets[x][i][0] for i in range(dataset_sizes[x])])
    for x in ["train", "val", "test"]}

labels = {x: np.array([datasets[x][i][1] for i in range(dataset_sizes[x])]).ravel()
            for x in ["train", "val", "test"]}

In [ ]:
print(inputs["train"][1].shape)


In [ ]:
from medmnist import INFO
info = INFO[data_flag]

class_names = info["label"]
class_names = {int(key): value for key, value in class_names.items()}

In [ ]:
clf = svm.SVC(kernel="rbf", C=1, class_weight=None, probability=True)

In [ ]:
clf.fit(np.concatenate((inputs["train"], inputs["val"]), axis=0), 
                    np.concatenate((labels["train"], labels["val"]), axis=0))

In [ ]:
clf.score(inputs["test"], labels["test"])

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, accuracy_score

# Get predictions and probabilities
predictions = clf.predict(inputs["test"])
probabilities = clf.predict_proba(inputs["test"])  # Ensure probability=True when training

# Calculate accuracy
acc = accuracy_score(labels["test"], predictions)

# Calculate confusion matrix
cm = confusion_matrix(labels["test"], predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")

# Calculate ROC AUC for multiclass (one-vs-rest strategy)
auc = roc_auc_score(labels["test"], probabilities, multi_class="ovr")

# Print results
print(f"Test Accuracy: {acc:.4f}")
print(f"Test AUC (OVR): {auc:.4f}")

# Show confusion matrix plot
plt.show()

In [ ]:
import joblib

filepath = os.path.join(os.getcwd(), "flatten_SVM.pkl")

data = {
    "model": clf,
    "hog": False,
}

joblib.dump(data, filepath)
print(f"Model saved to {filepath}")